In [ ]:
import win32com.client as client
import pandas as pd
import datetime as dt
import os

tabela = pd.read_excel('PARCELAS EM ATRASO - VIDA.xlsx')
display(tabela)

hoje = dt.datetime.now()

tabela_devedores = tabela.loc[tabela['Status']=='Em aberto']
tabela_devedores = tabela_devedores.loc[tabela_devedores['Data prevista para pagamento']<hoje]

outlook = client.Dispatch('Outlook.Application')

emissor = outlook.session.Accounts['seuemail@dominio.com']

dados = tabela_devedores[['Valor em aberto','Data prevista para pagamento','E-mail','N° Apólice','Cliente','N° Parcela']].values.tolist()

for dado in dados:
    destinatario = dado[2]
    apolice = dado[3]
    prazo = dado[1]
    prazo = prazo.strftime("%d/%m/%Y")
    valor = dado[0]
    cliente = dado[4]
    parcela = dado[5]
    assunto = 'Atraso de pagamento'
    mensagem = outlook.CreateItem(0)
    mensagem.Display()
    mensagem.To = destinatario
    mensagem.Subject = f'Lembrete sobre o pagamento de sua parcela - Seguro Vida  - Porto Seguros - Apólice n° {apolice} - {cliente} - {parcela}° parcela'
    corpo_mensagem = f'''
    Prezado Cliente,

    Espero que esta mensagem o encontre bem. Estamos entrando em contato para alertar sobre a parcela em atraso referente ao seu seguro de vida.

    É importante ressaltar que o não pagamento das parcelas pode acarretar no cancelamento do seu seguro, o que pode representar um grande risco para a sua proteção financeira e de seus beneficiários.

    Para evitar quaisquer inconvenientes e garantir a continuidade do seu seguro, pedimos que regularize o pagamento das parcelas pendentes o mais breve possível.
    {parcela}° parcela - Apólice {apolice}
    Segurado: {cliente}
    Valor em aberto: R${valor:.2f}

    Segue boleto atualizado em anexo.

    Caso já tenha sido efetuado o pagamento, pode desconsiderar esta mensagem.

    Atenciosamente,
    '''
    mensagem.Body = corpo_mensagem

    def criar_nome_pdf(cliente, parcela):
        return f'{cliente} - Parcela {parcela}.pdf'

    pasta_boletos = r"caminho/para/boletos"

    caminho_pdf = os.path.join(pasta_boletos, criar_nome_pdf(cliente, parcela))

    # Anexar o arquivo PDF ao e-mail
    anexo = mensagem.Attachments.Add(caminho_pdf)

    mensagem.oleobj.Invoke(*(64209,0,8,0,emissor))
    mensagem.Save()
    mensagem.Send()